In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, fcluster
import re
import numpy as np
import tqdm
from copy import deepcopy

stop_words = ['of', 'for', 'the', 'and']

In [2]:
def clean_text(text):
    # Replace '&' with 'and'
    text = text.replace('&', 'and')
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return text

In [4]:
# Load the CSV files
borrower_em = pd.read_csv('..\Data-In\Borrower_EM.csv')
borrower_wb = pd.read_csv('..\Data-In\Borrower_WB.csv')

In [5]:
borrower_wb.columns

Index(['PKGID', 'ORDER', 'BORROWERID', 'BORROWER', 'ABBR', 'COUNTRY',
       'COUNTRYISO3', 'TYPE', 'PURPOSE', 'ISIC'],
      dtype='object')

In [7]:
# Concatenating the two input files

# This variable is the EM file with empty TYPE and PURPOSE columns added  
borrower_em_with_type_purpose = borrower_em
borrower_em_with_type_purpose['TYPE'] = None
borrower_em_with_type_purpose['PURPOSE'] = None

In [81]:
borrower_combined = pd.concat([borrower_em_with_type_purpose, borrower_wb])
borrower_combined.reset_index(drop=True, inplace=True)

In [82]:
# Extract the borrower names
borrowers_augmented = deepcopy(borrower_combined)
borrowers_augmented.insert(0, 'OG_INDEX', borrowers_augmented.index)
borrowers_augmented.insert(5, 'CLEAN_BORROWER', borrowers_augmented['BORROWER'].apply(clean_text))

### Clustering

In [18]:
# Vectorize the borrower names using TF-IDF
vectorizer = TfidfVectorizer().fit_transform(borrowers_augmented['CLEAN_BORROWER'])
vectors = vectorizer.toarray()
vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
# Calculate cosine similarity matrix
cosine_sim_matrix = cosine_similarity(vectors)
cosine_sim_matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.]])

In [20]:
linkage_matrix = linkage(1 - cosine_sim_matrix, method='ward')

In [21]:
threshold = 0.35  
clusters = fcluster(linkage_matrix, threshold, criterion='distance')

###

In [83]:
borrowers_augmented.loc[:, 'CLUSTER'] = clusters

In [84]:
borrowers_augmented.columns

Index(['OG_INDEX', 'PKGID', 'ORDER', 'BORROWERID', 'BORROWER',
       'CLEAN_BORROWER', 'ABBR', 'COUNTRY', 'COUNTRYISO3', 'ISIC', 'TYPE',
       'PURPOSE', 'CLUSTER'],
      dtype='object')

In [70]:
def impute_mode(group, column):
    if not group[column].mode().empty:
        mode_value = group[column].mode().iloc[0]
    else:
        mode_value = ''  # Default value if mode is not available
    imputed = group[column].isnull()
    group[column] = group[column].fillna(mode_value)
    return group, imputed

### Dealing with problematic clusters (Multiple Borrower IDs in the same cluster)

In [85]:
problematic_clusters = borrowers_augmented.groupby('CLUSTER').filter(lambda x: x['BORROWERID'].nunique() > 1)

# We also need to get all other rows matching these borrowerids that the clusters haven;t picked up on
filtered_rows = borrowers_augmented[borrowers_augmented['BORROWERID'].isin(problematic_clusters['BORROWERID'].unique())]

problematic_clusters_complete = pd.DataFrame(filtered_rows)

Fixing BID-B1000: C.E.S.A manually

In [86]:
prob_clusters_fixed = deepcopy(problematic_clusters_complete)
prob_clusters_fixed.insert(4,'NEW_BORROWERID', prob_clusters_fixed['BORROWERID'])
prob_clusters_fixed.loc[prob_clusters_fixed['BORROWERID']=='B1000', 'NEW_BORROWERID'] = 'B1218'

Creating unique Borrower IDs for each ID-Country pair. These IDs are set to 5 digits as a flag for manual correction having been done. 

In [87]:
seen_pairs = {}

for index, row in prob_clusters_fixed.iterrows():
    borrowerid = row['NEW_BORROWERID']
    country = row['COUNTRYISO3']

    if borrowerid not in seen_pairs:
        seen_pairs[borrowerid] = {}

    if country not in seen_pairs[borrowerid]:
        seen_pairs[borrowerid][country] = len(seen_pairs[borrowerid]) + 1

    new_borrowerid = f'{borrowerid}{seen_pairs[borrowerid][country]}'
    prob_clusters_fixed.at[index, 'NEW_BORROWERID'] = new_borrowerid


Using the fixed clusters as reference and updating information in main dataframe.

In [88]:
for index, row in prob_clusters_fixed.iterrows():
    og_index = row['OG_INDEX']
    borrower_id_new = row['NEW_BORROWERID']
    borrowers_augmented.loc[borrowers_augmented['OG_INDEX']==og_index, 'BORROWERID'] = borrower_id_new

Filling in the missing values for ISIC, COUNTRYISO3, and COUNTRY

In [89]:
borrowers_augmented.loc[:, 'IMPUTED'] = 0

# Impute missing ISIC values and update the 'IMPUTED' column
for borrower_id, group in borrowers_augmented.groupby('BORROWERID'):
    imputed_isic = group['ISIC'].isnull()
    group, imputed_isic = impute_mode(group, 'ISIC')
    borrowers_augmented.loc[borrowers_augmented['BORROWERID'] == borrower_id, 'ISIC'] = group['ISIC']
    borrowers_augmented.loc[borrowers_augmented['BORROWERID'] == borrower_id, 'IMPUTED'] += imputed_isic.astype(int)

# Impute missing COUNTRYISO3 values and update the 'IMPUTED' column
for borrower_id, group in borrowers_augmented.groupby('BORROWERID'):
    imputed_country = group['COUNTRYISO3'].isnull()
    group, imputed_country = impute_mode(group, 'COUNTRYISO3')
    borrowers_augmented.loc[borrowers_augmented['BORROWERID'] == borrower_id, 'COUNTRYISO3'] = group['COUNTRYISO3']
    borrowers_augmented.loc[borrowers_augmented['BORROWERID'] == borrower_id, 'IMPUTED'] += 2 * imputed_country.astype(int)


In [90]:
def set_country_mode(group):
    mode_country = group['COUNTRY'].mode()[0]  # Get the mode of the COUNTRY column
    group['COUNTRY'] = mode_country
    return group

In [74]:
# Assign the mode of the country to all rows with same COUNTRYISO3 code
borrowers_augmented = borrowers_augmented.groupby('COUNTRYISO3').apply(set_country_mode)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10636\3631109854.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  borrowers_augmented = borrowers_augmented.groupby('COUNTRYISO3').apply(set_country_mode)


Filtering out all the rows with missing COUNTRYISO3 before filling in COUNTRY names

In [102]:
borrowers_augmented['COUNTRYISO3'].replace('', None, inplace=True)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10636\3367917712.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  borrowers_augmented['COUNTRYISO3'].replace('', None, inplace=True)


In [103]:
filtered_data = borrowers_augmented.dropna(subset=['COUNTRYISO3'])

# Apply the function to each group of COUNTRYISO3
filtered_data = filtered_data.groupby('COUNTRYISO3').apply(set_country_mode)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10636\3687408022.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_data = filtered_data.groupby('COUNTRYISO3').apply(set_country_mode)


In [104]:
updated_borrower_data = borrowers_augmented.merge(
    filtered_data[['OG_INDEX', 'COUNTRY']],
    on='OG_INDEX',
    how='left',
    suffixes=('', '_updated')
)

updated_borrower_data['COUNTRY'] = updated_borrower_data['COUNTRY_updated'].combine_first(updated_borrower_data['COUNTRY'])

updated_borrower_data.drop(columns=['OG_INDEX', 'COUNTRY_updated'], inplace=True)


### Exporting Result

In [105]:
final_borrower_data = updated_borrower_data.drop(columns=['CLEAN_BORROWER', 'CLUSTER'], inplace=False)

In [106]:
final_borrower_data.to_csv('..\Data-Out\Borrower_Combined_Result.csv')